In [2]:
from bs4 import BeautifulSoup
from lxml import etree as ET
import pandas as pd
import unicodedata
import matplotlib.pyplot as plt 
import plotly.express as px
import numpy as np
import polars as pl

In [4]:
dump = pl.read_parquet('data/online_diss_cleaned_03-2024.parquet') 
data = dump.to_pandas()
#data = pd.read_csv('data/check-data-csv.csv')
#data

In [5]:
#Get missing years from 008: 
data['Year'] = data['Year'].fillna(data['008'].str[7:11])

In [6]:
# Filter entries that don't have a publication date and reduce timespan: 

data1 = data[data['Year'].notna()]
data1.query("Year.str.len() == 4")

data1 = data1.astype({'Year':'int'})
data1 = data1[(data1['Year'] >= 1990) & (data1['Year'] <= 2023)]
data2 = data1[data1['Year'] >= 1990]

print("Data: ", len(data))
print("Data 1990-2023: ", len(data1))
print("Data >= 1990: ", len(data2))

Data:  324782
Data 1990-2023:  321558
Data >= 1990:  321558


In [7]:
#create new dataframe that only includes the publication year and value count: 
s = data1['Year'].value_counts().sort_index()

new = pd.DataFrame(s)
new = new.rename(columns={"Year": "count"})
new['years'] = new.index
new

,count,years
Year,,
1990,35,1990
1991,49,1991
1992,55,1992
1993,53,1993
1994,75,1994
1995,110,1995
1996,129,1996
1997,332,1997
1998,755,1998


In [8]:
#check data: 

df2 = new['count'].sum()
df2

321558

In [9]:
no_data = data[data['Year'].isnull()]
no_data

,ID,Creator,Title,lang,Place,Year,DDC,DDC1,DDC2,DDC-DE?,Sachgruppe,diss,notes,008


In [10]:
data1.loc[data1["Year"] == "unknown"]

,ID,Creator,Title,lang,Place,Year,DDC,DDC1,DDC2,DDC-DE?,Sachgruppe,diss,notes,008


In [11]:
new.to_json("data/diss_years.json", index=True)